In [7]:
## Decorators (contd.)

## Decorator Parameter

SyntaxError: invalid syntax (Temp/ipykernel_8852/1235331270.py, line 1)

In [ ]:
# demo for parameterized decorator
def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / reps
        print(avg_elapsed)
        return result
    return inner

In [ ]:
# put the above code inside an outer function
# timed here is called as "decorator factory" (outer function that returns the decorator)

def timed(reps):
    def dec(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / reps
            print(f'Avg run time: {avg_elapsed}')
            return result
        return inner
    return dec


In [ ]:
def calc_fib_recurse(n):
    return 1 if n<3 else calc_fib_recurse(n-1)+calc_fib_recurse(n-2)

In [ ]:
@timed(5)
def fib(n):
    return calc_fib_recurse(n)


In [ ]:
fib(28)

Avg run time: 0.1994881400000395


317811

In [ ]:
fib(28)

<function __main__.timed.<locals>.dec.<locals>.inner(*args, **kwargs)>

## Decorator Class

In [ ]:
# a simple decorator factory
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print(f'decorated function called: a = {a}, b = {b}')
            return fn(*args, **kwargs)
        return inner
    return dec

In [ ]:
@my_dec(10, 20)
def my_func(a):
    print(f'Hello {a}')

In [ ]:
my_func('Dhinesh')

decorated function called: a = 10, b = 20
Hello Dhinesh


In [ ]:
class MyClass:
    def __init__(self, a, b) -> None:
        self.a = a
        self.b = b

    def __call__(self, c):
        print(f'Caled a = {self.a}, b = {self.b}, c  = {c}')


In [ ]:
obj = MyClass(10, 20)

In [ ]:
obj

In [ ]:
obj.__call__(100)

Caled a = 10, b = 20, c  = 100


In [ ]:
obj(100)

Caled a = 10, b = 20, c  = 100


In [ ]:
# adding inner function to the __call__ method that will enable a class to be used as an decorator
class MyClass:
    def __init__(self, a, b) -> None:
        self.a = a
        self.b = b

    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f'decorated function called: a = {self.a}, b = {self.b}')
            return fn(*args, **kwargs)
        return inner

In [ ]:
# decorating using the class
@MyClass(10, 20)
def my_func(s):
    print(f'Hello {s}')

In [ ]:
my_func('Dhinesh')

decorated function called: a = 10, b = 20
Hello Dhinesh


In [ ]:
# another way
obj = MyClass(30, 20)

def my_func(s):
    print(f'Hello {s}')

my_func = obj(my_func) # this will add the class decorator to the function
my_func('Dhinesh')

decorated function called: a = 30, b = 20
Hello Dhinesh


## Decorating Classes than functions

In [ ]:
from fractions import Fraction
f = Fraction(2,3)
f

Fraction(2, 3)

In [ ]:
# adding additional attribute
Fraction.speak = 100
Fraction.speak

100

In [ ]:
Fraction.speak = lambda self, message: f'Franction says: {message}'

In [ ]:
f.speak('THis is a late parrot')

'Franction says: THis is a late parrot'

In [ ]:
# adding another attribute

Fraction.is_integral = lambda self: self.denominator == 1

In [ ]:
f1 = Fraction(64, 8)

In [ ]:
f1

Fraction(8, 1)

In [ ]:
f1.is_integral()

True

In [ ]:
def dec_speak(cls):
    cls.speak = lambda self, message: f'{self.__class__.__name__} says {message}'
    return cls

In [ ]:
Fraction = dec_speak(Fraction) # mutuating with new attribute

In [ ]:
f1 = Fraction(2, 3)
f1.speak('Hello')

'Fraction says Hello'

In [ ]:
# mututaing attribute to a custom class
class Person:
    pass

Person = dec_speak(Person)
p = Person()
p.speak('This works!!!')

'Person says This works!!!'

In [1]:
# another example

from datetime import datetime, timezone

def info(self):
    results = []
    results.append(f'time: {datetime.now(timezone.utc)}')
    results.append(f'Class {self.__class__.__name__}')
    results.append(f'id: {hex(id(self))}')

    for k,v in vars(self).items():
        results.append(f'{k}: {v}')
    
    return results

def debug_info(cls):
    cls.debug = info
    return cls 

In [2]:
@debug_info
class Person:
    def __init__(self, name, birth_year) -> None:
        self.name = name
        self.birth_year = birth_year

    def say_hi():
        return 'Hello there !'

In [3]:
p = Person('Dhinesh', 1989)
p.debug()

['time: 2021-12-26 07:04:45.615756+00:00',
 'Class Person',
 'id: 0x25078c16890',
 'name: Dhinesh',
 'birth_year: 1989']

In [3]:
# another class

@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed) -> None:
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError('Speed cannot exceed top_speed')
        else:
            self.speed = new_speed

In [4]:
favorite = Automobile('Ford', 'Model T', 1908, 45)

In [5]:
favorite.debug()

['time: 2021-12-26 07:07:30.679081+00:00',
 'Class Automobile',
 'id: 0x1782bde6440',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

In [ ]:
favorite.speed = 40

## Decorator application : Single Dispatch Generic Function

In [5]:
from html import escape 

def html_escape(arg):
    return escape(str(arg))


def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(k, v) for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'


In [3]:
print(html_str("""this is 
a multi line string
with special characters: 10 < 100"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &lt; 100


In [6]:
print(html_int(255))

255(<i>0xff</i>)


In [7]:
print(html_escape(10+2j))

(10+2j)


In [8]:
from decimal import Decimal
from html import escape

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [9]:
htmlize(100)

'100(<i>0x64</i>)'

In [11]:
print(htmlize([1,2,3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [12]:
print(htmlize(["""Python
rocks! 0<1
""", (10,20,30), 100]))

<ul>
<li>Python
rocks! 0&lt;1
</li>
<li>(10, 20, 30)</li>
<li>100</li>
</ul>


In [9]:
from decimal import Decimal
from html import escape

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg, set):
        return html_set(arg)
    else:
        return html_escape(arg)

In [10]:
def html_escape(arg):
    return escape(str(arg))


def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)

In [7]:
print(htmlize(["""Python
rocks! 0<1
""", (10,20,30), 100]))

<ul>
<li>Python<br/>
rocks! 0&lt;1<br/>
</li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>20(<i>0x14</i>)</li>
<li>30(<i>0x1e</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


In [12]:
htmlize({1,2,3})

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [13]:
def htmlize(arg):

    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_int,
        str: html_str,
        list: html_list,
        set: html_set,
        dict: html_dict
    }

    fn = registry.get(type(arg), registry[object])
    return fn(arg)


In [14]:
htmlize(-100)

'-100(<i>-0x64</i>)'

In [15]:
from functools import singledispatch

from numbers import Integral
from collections.abc import Sequence

@singledispatch
def htmlize(a):
    return escape(str(a))

In [16]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [17]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [18]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [19]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>})

In [20]:
htmlize.dispatch(int)

<function __main__.htmlize_integral_number(a)>

In [21]:
htmlize(10)

'10(<i>0xa</i>)'

In [22]:
@htmlize.register(Sequence)
def html_list(l):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [23]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [24]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [26]:
htmlize('python') # we will get RecursionError here

RecursionError: maximum recursion depth exceeded

In [27]:
@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [29]:
htmlize('python 1 < 100')

'python 1 &lt; 100'

In [30]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [31]:
@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return '({0})'.format(', '.join(items))

In [32]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>,
              collections.abc.Sequence: <function __main__.html_list(l)>,
              str: <function __main__.html_str(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [33]:
htmlize((1,2,3))

'(1, 2, 3)'

In [34]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [38]:
@htmlize.register(Integral)
def _(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

@htmlize.register(Sequence)
def _(l):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(str)
def _(s):
    return html_escape(s).replace('\n', '<br/>\n')